<a href="https://colab.research.google.com/github/ichiyan/brain-decoding/blob/master/Eval_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Inception Score: “How realistic is this image relative to the pretrained model?”

* FID Score: “How similar is this group of images relative to this other group of images?”

* LPIPS Score: “Did the structure of this patch change, and by how much?”

* SSIM, MSSIM, PSNR: “How noisy is this generated image as compared to the ground truth image?”

# Set-up installation

In [1]:
!git clone https://github.com/ichiyan/brain-decoding.git

Cloning into 'brain-decoding'...
remote: Enumerating objects: 513, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 513 (delta 7), reused 0 (delta 0), pack-reused 497
Receiving objects: 100% (513/513), 507.71 MiB | 13.49 MiB/s, done.
Resolving deltas: 100% (127/127), done.
Updating files: 100% (439/439), done.


In [2]:
cd /content/brain-decoding

/content/brain-decoding


In [ ]:
!git pull

Updating 94c1638..0ea2b40
Fast-forward
 scripts/latest/eval_extract_features.py   | 146 ++++++++++++++++++++++++++++++++++++++++++++++++++
 scripts/latest/evaluate_reconstruction.py |  93 ++++++++++++++++++++++++++++++++
 scripts/latest/save_test_images.py        |  16 ++++++
 3 files changed, 255 insertions(+)
 create mode 100644 scripts/latest/eval_extract_features.py
 create mode 100644 scripts/latest/evaluate_reconstruction.py
 create mode 100644 scripts/latest/save_test_images.py


In [ ]:
mv /content/brain-decoding/scripts/latest/evaluate_reconstruction.py /content

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torchmetrics torch torch-fidelity ipyplot torchvision webdataset clip clip-retrieval

In [ ]:
!pip install clip-retrieval

# Imports

In [5]:
from PIL import Image as ImagePIL
import os, sys
import cv2
import numpy as np

In [6]:
import torch
from torchmetrics.image import StructuralSimilarityIndexMeasure
import torchvision.transforms as transforms

In [7]:
import h5py
import scipy.io as spio
import nibabel as nib
import torchvision
import torchvision.models as tvmodels
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
from PIL import Image
import clip

import skimage.io as sio
from skimage import data, img_as_float
from skimage.transform import resize as imresize
from skimage.metrics import structural_similarity as ssim
import scipy as sp

In [8]:
import argparse

In [9]:
def get_default_device():
    """Picking GPU if available or else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
device = get_default_device()

# Convert imgs to npy

In [ ]:
'''
Converts all images in a directory to '.npy' format.
Use np.save and np.load to save and load the images.
Use it for training your neural networks in ML/DL projects.
'''

# Path to image directory
path = "/content/drive/MyDrive/brain_decoding/data/reconstructed/425_stim_53l/"
dirs = os.listdir( path )
dirs.sort()
x_train=[]

def load_dataset():
    # Append images to a list
    for item in dirs:
      # print(os.path.isfile(path+item))
      if os.path.isfile(path+item):
        im = pImage.open(path+item).convert("RGB")
        im = np.array(im)
        x_train.append(im)
    # print(x_train)

In [ ]:
load_dataset()

# Convert and save the list of images in '.npy' format
imgset=np.array(x_train)
np.save("425recons.npy",imgset)

In [ ]:
sub=1

In [ ]:
recon_imgs = np.load('/content/recons.npy'.format(sub,sub)).astype(np.uint8)

In [ ]:
recon_imgs1 = np.load('/content/425recons.npy'.format(sub,sub)).astype(np.uint8)

In [ ]:
print(recon_imgs1.shape)

(982, 425, 425, 3)


In [ ]:
print(recon_imgs.shape)
print(recon_imgs1.shape)

(982, 512, 512, 3)
(149, 425, 425, 3)


In [ ]:
ground = ImagePIL.fromarray(gt_stim[ndx])

In [ ]:
cp /content/425recons.npy /content/drive/MyDrive/brain_decoding/data/reconstructed/recons425.npy

# Convert to numpy

# Show image

In [ ]:
sub = 1
ndx=115
layerA = 53
layerB = 41

In [ ]:
gt_stim = np.load('/content/drive/MyDrive/brain_decoding/data/fMRI/NSD/nsd_test_stim_sub1.npy'.format(sub,sub)).astype(np.uint8)
recons_imgs = np.load('/content/drive/MyDrive/brain_decoding/data/reconstructed/53l/recons.npy'.format(sub,sub)).astype(np.uint8)
recons425_imgs = np.load('/content/drive/MyDrive/brain_decoding/data/reconstructed/recons425.npy'.format(sub,sub)).astype(np.uint8)

In [ ]:
print(gt_stim[ndx].shape)
print(recons_imgs[ndx].shape)
print(recons425_imgs[ndx].shape)

(425, 425, 3)
(512, 512, 3)
(425, 425, 3)


In [ ]:
# img1 = Image('/content/drive/MyDrive/brain_decoding/data/reconstructed/{}l/{}.png'.format(layerA, ndx), width=300)
#img2 = Image('/content/drive/MyDrive/brain_decoding/data/reconstructed/{}l/{}.png'.format(layerB, ndx), width=300)

In [ ]:
img1 = ImagePIL.open('/content/drive/MyDrive/brain_decoding/data/reconstructed/{}l/{}.png'.format(layerA, ndx))
img2 = ImagePIL.open('/content/drive/MyDrive/brain_decoding/data/reconstructed/{}l/{}.png'.format(layerB, ndx))
ground = ImagePIL.fromarray(gt_stim[ndx])
img3 = ImagePIL.fromarray(recons425_imgs[ndx])

In [ ]:
print("Ground Truth")
display(ground)
print(layerA," Layers")
display(img1)
print(layerB," Layers")
display(img2)

# SSIM

*   preds – estimated image
*   target – ground truth image

[SSIM doc](https://torchmetrics.readthedocs.io/en/stable/image/structural_similarity.html)

Access images

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
test = torch.from_numpy(gt_stim)
type(test)

torch.Tensor

In [ ]:
# preds = torch.rand([3, 3, 256, 256])
# target = preds * 0.75
preds = torch.from_numpy(recons425_imgs)
# transform(img3).unsqueeze(0)
target = torch.from_numpy(gt_stim)
# transform(ground).unsqueeze(0)
print(type(preds))
print(preds.shape)
print("ground: ", target.shape)

<class 'torch.Tensor'>
torch.Size([982, 425, 425, 3])
ground:  torch.Size([982, 425, 425, 3])


In [ ]:
preds[0]

Call SSIM

In [ ]:
ssim = StructuralSimilarityIndexMeasure(data_range=1.0)
ssim(preds, target)

RuntimeError: ignored

# FID

In [ ]:
import torch
_ = torch.manual_seed(123)
from torchmetrics.image.fid import FrechetInceptionDistance
import torch_fidelity
import tensorflow as tf

In [ ]:
torch_fidelity.__version__

'0.3.0'

In [ ]:
fid = FrechetInceptionDistance(feature=64)

Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:00<00:00, 388MB/s]


In [ ]:
dataset_ground = tf.data.Dataset.from_tensor_slices(gt_stim)
dataset_ground

<_TensorSliceDataset element_spec=TensorSpec(shape=(425, 425, 3), dtype=tf.uint8, name=None)>

In [ ]:
dataset_recon = tf.data.Dataset.from_tensor_slices(recons425_imgs)
dataset_recon

<_TensorSliceDataset element_spec=TensorSpec(shape=(425, 425, 3), dtype=tf.uint8, name=None)>

In [ ]:
# generate two slightly overlapping image intensity distributions
imgs_dist1 = torch.randint(0, 200, (100, 3, 299, 299), dtype=torch.uint8)
imgs_dist2 = torch.randint(100, 255, (100, 3, 299, 299), dtype=torch.uint8)

In [ ]:
fid.update(imgs_dist1, real=True)
fid.update(imgs_dist2, real=False)
fid.compute()

tensor(12.7202)

In [ ]:
metrics_dict = torch_fidelity.calculate_metrics(
    input1=dataset_recon,
    input2=dataset_ground,
    cuda=True,
    isc=True,
    fid=True,
    kid=True,
    prc=True,
    verbose=False,
)

ValueError: ignored

# FID [pytorch]

Since we are under the required 2048 images, select the dimensionality of features to use with the flag --dims N, where N is the dimensionality of features. The choices are:

* 64: first max pooling features
* 192: second max pooling features
* 768: pre-aux classifier features
* 2048: final average pooling features (this is the default)

In [ ]:
!pip install pytorch-fid

In [ ]:
!python -m pytorch_fid /content/drive/MyDrive/brain_decoding/data/reconstructed/41l/stim_41l /content/drive/MyDrive/brain_decoding/data/reconstructed/53l/stim_53l --dims 768

100% 20/20 [00:06<00:00,  3.27it/s]
100% 20/20 [00:06<00:00,  3.31it/s]
FID:  0.09330198096745956


# BrainDiffuser

<span style="background:Green">
Quantitative Evaluation
Although results are expected to be similar, it may vary because of variations at reconstruction


*   Save test images to directory ```python scripts/save_test_images.py```
*  Extract evaluation features for test images using
```
# python scripts/eval_extract_features.py -sub 0
```
*   Extract evaluation features for reconstructed images of any subject using ```python scripts/eval_extract_features.py -sub x```
*   Obtain quantitative metric results for each subject using ```python scripts/evaluate_reconstruction.py -sub x```
</span>

In [10]:
cd /content/brain-decoding

/content/brain-decoding


<h2>From npy to images</h2>

In [12]:
images = np.load('/content/drive/MyDrive/brain_decoding/data/fMRI/NSD/nsd_test_stim_sub1.npy')
#PATH WHERE IMAGES GET SAVED TO
test_images_dir = '/content/brain-decoding/data/nsddata_stimuli/stimuli'

if not os.path.exists(test_images_dir):
   os.makedirs(test_images_dir)
for i in range(len(images)):
    im = ImagePIL.fromarray(images[i].astype(np.uint8))
    im.save('{}/{}.png'.format(test_images_dir,i))

<h2>Extract Features</h2>

In [13]:
print(torch.cuda.is_available())

True


Changes made
* device=0 [since there's only 1 gpu]
* replace `images_dir` & `feats_dir`

In [ ]:
!python /content/brain-decoding/scripts/latest/eval_extract_features.py -sub 0

In [16]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [17]:
get_available_gpus()

['/device:GPU:0']

<h2>Evaluate Reconstruction</h2>

* change `images_dir` & `feats_dir` at line50
* double check the paths used for `gen_image` & `gt_image` at line 83

In [21]:
!python /content/brain-decoding/scripts/latest/editted_evaluate_reconstruction.py -sub 1

/content/brain-decoding/scripts/latest/editted_evaluate_reconstruction.py:91: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_res = ssim(gen_image, gt_image, multichannel=True, gaussian_weights=True, sigma=1.5, use_sample_covariance=False, data_range=1.0)
PixCorr: 0.031211689812505133
SSIM: 0.34881656110681275


In [ ]:
cp /content/evaluate_reconstruction.py /content/drive/MyDrive/brain_decoding/editted_evaluate_reconstruction.py

# MindEye


Evaluating Reconstructions
After you have saved a .pt file from running `Reconstructions.py` or `Retrievals.py`, you can use `Reconstruction_Metrics.py` to evaluate reconstructed images using the same low- and high-level image metrics used in the paper.

* Set `recon_path` to the name of the file in "fMRI-reconstruction-NSD/src" that was output from *Reconstructions.py* (should be `{model_name}_recons_img2img{img2img_strength}_{recons_per_sample}samples.pt`).
* Alternatively, to evaluate LAION-5B retrievals, you can replace recon_path with the name of the .pt file output from *Retrievals.py* (should be `{model_name}_laion_retrievals_top16.pt`).
* Set `all_images_path` to the all_images.pt file in "fMRI-reconstruction-NSD/src" that was output from either *Reconstructions.py* or *Retrievals.py* (should be `all_images.pt`).

```
$ python Reconstruction_Metrics.py --help
```

```
usage: Reconstruction_Metrics.py [-h] [--recon_path RECON_PATH]
                                 [--all_images_path ALL_IMAGES_PATH]

Model Training Configuration

options:
  -h, --help            show this help message and exit
  --recon_path RECON_PATH
                        path to reconstructed/retrieved outputs
  --all_images_path ALL_IMAGES_PATH
                        path to ground truth outputs
```

In [ ]:
!pip install open_clip_torch dalle2_pytorch typing-extensions

In [ ]:
!pip install typing-extensions --upgrade

In [ ]:
!pip install diffusers

In [ ]:
cp /content/drive/MyDrive/brain_decoding/clip_client.py /content/brain-decoding/mindeye/src/clip_client.py

In [ ]:
import sys
sys.path.insert(0,'/content/brain-decoding/mindeye/src')

In [ ]:
import clip_retrieval

In [ ]:
!python --version

Python 3.10.12


In [ ]:
import webdataset, sys

import utils

In [ ]:
!python /content/brain-decoding/mindeye/src/Reconstruction_Metrics.py --recon_path '/content/drive/MyDrive/brain_decoding/data/reconstructed/recons425.npy' --all_images_path '/content/drive/MyDrive/brain_decoding/data/fMRI/NSD/nsd_train_stim_sub1.npy'

device: cuda
ViT-L/14 cuda
Traceback (most recent call last):
  File "/content/brain-decoding/mindeye/src/Reconstruction_Metrics.py", line 89, in <module>
    all_brain_recons = torch.load(f'{recon_path}')
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 1028, in load
    return _legacy_load(opened_file, map_location, pickle_module, **pickle_load_args)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 1246, in _legacy_load
    magic_number = pickle_module.load(f, **pickle_load_args)
_pickle.UnpicklingError: STACK_GLOBAL requires str


# Test

In [ ]:
ckpt_drive_path = os.path.join('/content/drive/MyDrive/brain_decoding/data/mindeye low-level/ckpt/imgds.txt')


In [ ]:
x = torch.tensor([0, 1, 2, 3, 4])
torch.save(x, ckpt_drive_path)